In [1]:
import os
import os.path as osp
import cv2
import shutil
import random
from tqdm.notebook import tqdm
from augmentations import augmentation_choices, transform_image

In [ ]:
# augmentation_choices = [
#                   'gaussian', 'localvar', 'poisson', 'salt', 'pepper', 's&p', 'speckle', 'affine', # skimage
#                   'rain', 'shadow', 'flare', 'snow' # Albumentations
#                   ]
augmentation_choices = ['gaussian', 'rain', 'shadow', 'flare', 'snow']

In [ ]:
# In this cell, define a function to save the NEW split files
# Save a split file for EACH augmentation
# and one for the whole augmented dataset

In [ ]:
def make_subdir(subdir):
    if not osp.isdir(subdir):
        try:
            os.mkdir(subdir)
        except FileNotFoundError:
            print("Make sure the parent folder exists")

In [5]:
split_file = 'kitti_eigen_test.txt'
input_prefix_path = "E:\Thesis_Datasets\\kitti_full\\"
input_depth_dir = 'val'
valid_image_count = 0
img_infos = []
output_prefix_path = "E:\Thesis_Datasets\\Augmented_Images\\"
output_image_folder = "Data\\"
output_depth_folder = "Depth\\"

if split_file is not None:
    with open(split_file) as f:
        for line in tqdm(f, desc="Augmenting input files..."):
            img_name = line.strip().split(" ")[0]
            img_name = osp.join(input_prefix_path, img_name)
            depth_map = line.strip().split(" ")[1]
            depth_map = osp.join(input_prefix_path, input_depth_dir, depth_map)
            
            if osp.isfile(img_name) and osp.isfile(depth_map):
                valid_image_count += 1
                
                # Ensure the folder exists
                if not osp.isdir(osp.join(output_prefix_path, output_depth_folder)):
                    try:
                        os.mkdir(osp.join(output_prefix_path, output_depth_folder))
                    except FileNotFoundError:
                        print("Make sure the parent folder exists")
                
                #   Copy depth map
                shutil.copy(depth_map,
                            osp.join(output_prefix_path, output_depth_folder, depth_map.split('/')[-1]))
                            
                #   For each augmentation type:
                for aug in augmentation_choices:
                    if not osp.isdir(osp.join(output_prefix_path, output_image_folder, aug)):
                        try:
                            os.mkdir(osp.join(output_prefix_path, output_image_folder, aug))
                        except FileNotFoundError:
                            print("Make sure the parent folder exists")
                            
                    # if aug = affine, depth map needs to be augmented with the same scale, rotation and shear
                    if aug == 'affine':
                        scale=(1.1, 1.1)
                        rotation=random.uniform(-0.05, 0.05)
                        shear=random.uniform(-0.1, 0.1)
                        
                        img = transform_image(aug, cv2.imread(img_name), scale, rotation, shear)
                        augmented_depth = transform_image(aug, cv2.imread(depth_map), scale, rotation, shear)
                        
                        cv2.imwrite(osp.join(output_prefix_path, 
                                         output_image_folder, 
                                         aug, 
                                         img_name[img_name.rfind('/')+1:-4] + "_affine.png"),
                                    augmented_depth)
                        # Save augmented depth file as <depth/xxxxxx_affine.png> 
                        # insert [rgb_file, depth_file, augmentation_type] into LIST

                    else:
                        img = transform_image(aug, cv2.imread(img_name))

                    # Save file with new name in /<augmentation_type>/ folder
                    cv2.imwrite(osp.join(output_prefix_path, 
                                         output_image_folder, 
                                         aug, 
                                         img_name.split('/')[-1]),
                                img)
                #      insert [rgb_file, depth_file, augmentation_type] into LIST
            else:
                print("-----")
                print(img_name, " exists: ", osp.isfile(img_name))
                print(depth_map, " exists:", osp.isfile(depth_map))
                
            if valid_image_count > 10:
                break
            
            
            

print(f"Lines: {valid_image_count}")


Augmenting input files...: 0it [00:00, ?it/s]

-----
E:\Thesis_Datasets\kitti_full\2011_09_26/2011_09_26_drive_0002_sync/image_02/data/0000000075.png  exists:  True
E:\Thesis_Datasets\kitti_full\val\None  exists: False
-----
E:\Thesis_Datasets\kitti_full\2011_09_26/2011_09_26_drive_0002_sync/image_02/data/0000000072.png  exists:  True
E:\Thesis_Datasets\kitti_full\val\None  exists: False
-----
E:\Thesis_Datasets\kitti_full\2011_09_26/2011_09_26_drive_0002_sync/image_02/data/0000000003.png  exists:  True
E:\Thesis_Datasets\kitti_full\val\None  exists: False
Lines: 11


In [ ]:
print(osp.join(output_prefix_path, 
                                             output_image_folder, 
                                             aug, 
                                             img_name[img_name.rfind('/')+1:]))

In [4]:
osp.join(output_prefix_path, 
                                         output_image_folder, 
                                         aug, 
                                         img_name[img_name.rfind('/')+1:-4]+"_affine.png")

'E:\\Thesis_Datasets\\Augmented_Images\\Data\\snow\\0000000039_affine.png'

In [ ]:
depth_map[92:]

In [ ]:
print(depth_map[depth_map.rfind('/')+1:])

In [ ]:
print(osp.join(prefix_path, depth_map))
print(osp.isfile(osp.join(prefix_path, img_name)))

In [ ]:
# print(osp.isfile(osp.join("e:\Thesis_Datasets\\kitti_full\\", "2011_09_26/2011_09_26_drive_0001_sync/image_00", "timestamps.txt")))
print((osp.join("E:\Thesis_Datasets\\kitti_full\\", "2011_09_26\\2011_09_26_drive_0001_sync\\image_00", split_file)))

In [ ]:
img_info = dict()
            if depth_dir is not None: # benchmark test or unsupervised future
                depth_map = line.strip().split(" ")[1]                        
                if (depth_map == 'None') or not osp.isfile(osp.join(depth_dir, depth_map)):
                    continue
                img_info['ann'] = dict(depth_map=depth_map)
            img_name = line.strip().split(" ")[0]
            img_info['filename'] = img_name
            img_infos.append(img_info)